# Pricing Uncertainty Induced by Climate Change
by Professor [Michael Barnett](https://sites.google.com/site/michaelduglasbarnett/home), Professor [William Brock](https://www.ssc.wisc.edu/~wbrock/) and Professor [Lars Peter Hansen](https://larspeterhansen.org/), you could find the latest draft [here](https://larspeterhansen.org/research/papers/).

## Overview

This notebook is to provide the source code and explanations on how we solve our climate problem under __preference damage setting__. 


## Preliminary work: Importing required packages, importing parameters setting, defining functions

In [1]:
# Required packages
import os
import sys
sys.path.append('./src')
from supportfunctions import *
sys.stdout.flush()

## Solving HJB for Preference Damage

In [6]:
# Loading Smart guesses
guess = pickle.load(open('./data/WeightedAverseguess.pickle', "rb", -1))
v0_guess = guess['v0']
q_guess = guess['q']
e_guess = guess['e']

Here we outline the procedure used in the code to solve this HJB. We write out the steps here, and then refer to those steps in the code below, more details could be found in the remarks part

To solve the nonlinear partial differential equations that characterize the HJB equations for the planner's problems from our model, we use a so-called implicit, finite-difference scheme and a conjugate gradient method. Consultations with Joseph Huang, Paymon Khorrami and Fabrice Tourre played an important role in the software implementation. We briefly outline the steps to this numerical solution method below.

Recall that the HJB  equation  of interest for the consumption damages model  includes both minimization and maximization:
\begin{align*} 0 = \max_{a \in {\mathbb A}} \min_{q > 0, \int q P(d\theta) =1 } \min_{g \in {\mathbb R}^m} & - \delta V(x)  + \delta (1 - \kappa) \left[ \log \left( {\alpha}  - i - j \right)
+ k -  d   \right] + \delta \kappa \left( \log e +  r \right) \cr &
+ {\frac {\partial V}{\partial x}} (x) \cdot \left[\int_\Theta  \mu_X(x,a \mid \theta) q(\theta) P(d\theta)  + \sigma_X(x) g\right] \cr &
+{\frac 1 2} \textrm{trace} \left[\sigma_X(x)' {\frac {\partial^2 V}{\partial x \partial x'}}(x) \sigma_X(x) \right] \cr &
+ {\frac {\xi_m} 2} g'g + \xi_p \int_\Theta [\log q(\theta)]  q(\theta) P(d \theta).
\end{align*}

We proceed recursively as follows:


1) start with a value function guess ${\widetilde V}(x)$ and a decision function ${\widetilde a}(x)$;

2) given $({\widetilde V}, {\widetilde a})$, solve the minimization problem embedded in the HJB equation and produce an exponentially-tilted  density ${\widehat  q}$ and drift distortion ${\widehat g}$  conditioned on $x$ and  using the  approach described   in section D;

3) compute the implied relative entropy from the change in prior:
$$
{\widehat {\mathbb I}}(x) = \int_\Theta [\log {\widehat q}(\theta)]  {\widehat q}(\theta) P(d \theta);
$$

4)  solve the following maximization problem by choice of $a=(i,j,e)$:

\begin{align*}
& \delta (1 - \kappa)  \log \left( {\alpha}  - i - j \right)
 + \delta \kappa  \log e   \cr &
+ {\frac {\partial V}{\partial x}} (x) \cdot \int_\Theta  \mu_X\left(x,a   \mid \theta \right) {\widehat q}(\theta \mid x ) P(d\theta);
\end{align*}

   a) Compute ${\hat i}$ and ${\hat j}$ by solving the two first-order conditions for $i$ and $j$ with cobweb-style iterations.  Cobweb iterations converge or diverge depending the relative slopes of supply and demand functions.  By shrinking the step size, these slopes can be altered.

Expand the two equation system by adding a third equation that defines  a common "price" $p$,
$$
p =  {\frac {\delta (1-\kappa)}  {\alpha  - i - j} } = g(i+j).
$$
Write the two first-order conditions as
$$
p = {\frac {\phi_0 \phi_1 V_k(x) }{ 1 + \phi_1 i}} = f_1(i)
$$
$$
p  = V_r(x)  \left(  {\psi_0 \psi_1} \right) j^{\psi_1 - 1}  \exp\left[ \psi_1(k -  r)\right]  = f_2(j).
$$

Given $p$  and for step size $\tilde{\epsilon}$, compute

* $i^* = (f_1)^{-1}(p)$

* $j^* = (f_2)^{-1}(p)$

* $p^* = {\eta} g(i^* + j^*) + \left(1 - {\eta} \right) p$

* set $p  = p^*$.

Iterate to convergence.


b) Compute ${\hat e}$ by solving the first-order conditions
$$
{\frac {\delta \kappa} e} +  {\frac d {d e}} \left[V_x(x)  \cdot \int_\Theta  \mu_X\left(x, i, j, a  \mid \theta \right) {\widehat q}(\theta \mid x ) P(d\theta)\right] = 0 .
$$
These first-order conditions turn out not to depend on $(i,j)$.


5) use the minimization output from step (2) and  maximization output from step (4) and construct an adjusted drift using the following formula, which is the analog to formula (20) from the paper:
\begin{equation*}
{\widehat \mu}(x) = \int_\Theta  \mu_X\left(x, {\widehat a}  \mid \theta \right) {\widehat q}(\theta \mid x ) P(d\theta) + \sigma_X(x) {\widehat g}(x);
\end{equation*}






6)  construct the linear equation system for a new value function $V = {\widehat V}$:
\begin{align*}
0 =  & - \delta V(x)  + \delta (1 - \kappa) \left( \log \left[ {\alpha}  - {\widehat i}(x)  - {\widehat j} (x)  \right]
+ k -  d   \right) + \delta \kappa \left[ \log {\widehat e}(x)   +  r \right] \cr
& + {\frac {\partial V}{\partial x}} (x) \cdot {\widehat \mu}(x)
+{\frac 1 2} \textrm{trace} \left[\sigma_X(x)' {\frac {\partial^2 V}{\partial x \partial x'}}(x) \sigma_X(x) \right] \cr
& + {\frac {\xi_m} 2} {\widehat g}(x) \cdot {\widehat g}(x)   + \xi_p {\widehat {\mathbb I}}(x);
\end{align*}

7) modify this equation by adding a so-called "false transient" to the left-hand side:
\begin{align} \label{modify_linear}
{\frac {V(x) - {\widetilde V}(x)} \epsilon}  =  & - \delta V(x)  + \delta (1 - \kappa) \left( \log \left[ {\alpha}  - {\widehat i}(x)  - {\widehat j} (x)  \right]
+ k -  d    \right) + \delta \kappa \left[ \log {\widehat e}(x)   +  r \right] \cr
& + {\frac {\partial V}{\partial x}} (x) \cdot {\widehat \mu}(x)
+{\frac 1 2} \textrm{trace} \left[\sigma_X(x)' {\frac {\partial^2 V}{\partial x \partial x'}}(x) \sigma_X(x) \right] \cr
& + {\frac {\xi_m} 2} {\widehat g}(x) \cdot {\widehat g}(x)   + \xi_p {\widehat {\mathbb I}}(x);
\end{align}

8) solve the linear system from step (7) for $V= {\widehat V}$ using a conjugate-gradient method;

9) set ${\widetilde V} = {\widehat V}$ and ${\widetilde a} = {\widehat a}$ and repeat steps (2) - (8)  until convergence.



In [7]:
start_time = time.time()

# Damage function choices
damageSpec = 'Weighted'  # Choose among "High"(Weitzman), 'Low'(Nordhaus) and 'Weighted'

if damageSpec == 'High':
    weight = 0.0
elif damageSpec == 'Low':
    weight = 1.0
else:
    weight = 0.5

ξₚ =  1 / 4000  # Ambiguity Averse Paramter 
# Sensible choices are from 0.0002 to 4000, while for parameters input over 0.01 the final results won't alter as much
    
McD = np.loadtxt('./data/TCRE_MacDougallEtAl2017_update.txt')
par_lambda_McD = McD / 1000

β𝘧 = np.mean(par_lambda_McD)  # Climate sensitivity parameter, MacDougall (2017)
σᵦ = np.var(par_lambda_McD, ddof = 1)  # varaiance of climate sensitivity parameters
λ = 1.0 / σᵦ 

# Parameters are defined the same as they are in the paper
δ = 0.01        
κ = 0.032       
σ𝘨 = 0.02
σ𝘬 = 0.0161
σ𝘳 = 0.0339 
α = 0.115000000000000
ϕ0 = 0.0600
ϕ1 = 16.666666666666668
μ̄ₖ = -0.034977443912449
ψ0 = 0.112733407891680
ψ1 = 0.142857142857143

# parameters for damage function settings
power = 2 
γ1 = 0.00017675
γ2 = 2. * 0.0022
γ2_plus = 2. * 0.0197
γ̄2_plus = weight * 0 + (1 - weight) * γ2_plus

σ1 = 0
σ2 = 0
ρ12 = 0
F̄ = 2
crit = 2
F0 = 1

xi_d = -1 * (1 - κ)

# Regarding the choice of ε and η please refers to Remark 1
# False Trasient Time step
ε = 0.1
# Cobweb learning rate
η = 0.05


# Specifying Tolerance level
tol = 1e-8

# Grids Specification

# Coarse Grids
# nR = 30
# nF = 40
# nK = 25
# R = np.linspace(R_min, R_max, nR)
# F = np.linspace(F_min, F_max, nF)
# K = np.linspace(K_min, K_max, nK)

# hR = R[1] - R[0]
# hF = F[1] - F[0]
# hK = K[1] - K[0]

# Dense Grids

R_min = 0
R_max = 9
F_min = 0
F_max = 4000
K_min = 0
K_max = 18

hR = 0.05
hF = 25
hK = 0.15

R = np.arange(R_min, R_max + hR, hR)
nR = len(R)
F = np.arange(F_min, F_max + hF, hF)
nF = len(F)
K = np.arange(K_min, K_max + hK, hK)
nK = len(K)

# Here's how we discretize our state space formulating PDE, see Remark 2
(R_mat, F_mat, K_mat) = np.meshgrid(R,F,K, indexing = 'ij')
stateSpace = np.hstack([R_mat.reshape(-1,1,order = 'F'),F_mat.reshape(-1,1,order = 'F'),K_mat.reshape(-1,1,order = 'F')])

# Inputs for function quad_int; Integrating across parameter distribution
quadrature = 'legendre'
n = 30
a = β𝘧 - 5 * np.sqrt(σᵦ)
b = β𝘧 + 5 * np.sqrt(σᵦ)

v0 = κ * R_mat + (1-κ) * K_mat - β𝘧 * F_mat

episode = 1
FC_Err = 1
v0 = v0_guess
q = q_guess
e_star = e_guess

while episode == 0 or FC_Err > tol:
    vold = v0.copy()
    # Applying finite difference scheme to the value function
    v0_dr = finiteDiff(v0,0,1,hR) 
    v0_df = finiteDiff(v0,1,1,hF)
    v0_dk = finiteDiff(v0,2,1,hK)

    v0_drr = finiteDiff(v0,0,2,hR)
    v0_drr[v0_dr < 1e-16] = 0
    v0_dr[v0_dr < 1e-16] = 1e-16
    v0_dff = finiteDiff(v0,1,2,hF)
    v0_dkk = finiteDiff(v0,2,2,hK)

    if episode == 0:
        # First time into the loop
        B1 = v0_dr - xi_d * (γ1 + γ2 * F_mat * β𝘧 + γ2_plus * (F_mat * β𝘧 - F̄) ** (power - 1) * (F_mat >= (crit / β𝘧))) * β𝘧 * np.exp(R_mat) - v0_df * np.exp(R_mat)
        C1 = - δ * κ
        e = -C1 / B1
        e_hat = e
        Acoeff = np.ones(R_mat.shape)
        Bcoeff = ((δ * (1 - κ) * ϕ1 + ϕ0 * ϕ1 * v0_dk) * δ * (1 - κ) / (v0_dr * ψ0 * 0.5) * np.exp(0.5 * (R_mat - K_mat))) / (δ * (1 - κ) * ϕ1)
        Ccoeff = -α  - 1 / ϕ1
        j = ((-Bcoeff + np.sqrt(Bcoeff ** 2 - 4 * Acoeff * Ccoeff)) / (2 * Acoeff)) ** 2
        i = α - j - (δ * (1 - κ)) / (v0_dr * ψ0 * 0.5) * j ** 0.5 * np.exp(0.5 * (R_mat - K_mat))
        q = δ * (1 - κ) / (α - i - j)
    else:
        e_hat = e_star
        
        # Step 4 (a) : Cobeweb scheme to update i and j; q is an intermediary variable that determines i and j
        Converged = 0
        nums = 0
        while Converged == 0:
            i_star = (ϕ0 * ϕ1 * v0_dk / q - 1) / ϕ1
            j_star = (q * np.exp(ψ1 * (R_mat - K_mat)) / (v0_dr * ψ0 * ψ1)) ** (1 / (ψ1 - 1))
            if α > np.max(i_star + j_star):
                q_star = η * δ * (1 - κ) / (α - i_star - j_star) + (1 - η) * q
            else:
                q_star = 2 * q
            if np.max(abs(q - q_star) / η) <= 1e-5:
                Converged = 1
                q = q_star
                i = i_star
                j = j_star
            else:
                q = q_star
                i = i_star
                j = j_star
            
            nums += 1
        print('Cobweb Passed, iterations: {}, i error: {:10f}, j error: {:10f}'.format(nums, np.max(i - i_star), np.max(j - j_star)))

    a1 = np.zeros(R_mat.shape)
    b1 = xi_d * e_hat * np.exp(R_mat) * γ1
    c1 = 2 * xi_d * e_hat * np.exp(R_mat) * F_mat * γ2 
    λ̃1 = λ + c1 / ξₚ
    β̃1 = β𝘧 - c1 * β𝘧 / (ξₚ * λ̃1) -  b1 /  (ξₚ * λ̃1)
    I1 = a1 - 0.5 * np.log(λ) * ξₚ + 0.5 * np.log(λ̃1) * ξₚ + 0.5 * λ * β𝘧 ** 2 * ξₚ - 0.5 * λ̃1 * (β̃1) ** 2 * ξₚ
    #     R1 = \xi\_p.*(I1-(a1+b1.*β̃1+c1./2.*(β̃1).^2+c1./2./\lambda\tilde_1));
    R1 = 1 / ξₚ * (I1 - (a1 + b1 * β̃1 + c1 / 2 * β̃1 ** 2 + c1 / 2 / λ̃1))
    J1_without_e = xi_d * (γ1 * β̃1 + γ2 * F_mat * (β̃1 ** 2 + 1 / λ̃1)) * np.exp(R_mat)

    π̃1 = weight * np.exp(-1 / ξₚ * I1)

    # Step (2), solve minimization problem in HJB and calculate drift distortion, see remark 3 for more details
    def scale_2_fnc(x):
        return np.exp(-1 / ξₚ * xi_d * (γ1 * x + γ2 * x ** 2 * F_mat + γ2_plus * x * (x * F_mat - F̄) ** (power - 1) * ((x * F_mat - F̄) >= 0)) * np.exp(R_mat) * e_hat)  * norm.pdf(x,β𝘧,np.sqrt(σᵦ))

    scale_2 = quad_int(scale_2_fnc, a, b, n, 'legendre')

    def q2_tilde_fnc(x):
        return np.exp(-1 / ξₚ * xi_d * (γ1 * x + γ2 * x ** 2 * F_mat + γ2_plus * x * (x * F_mat - F̄) ** (power - 1) * ((x * F_mat - F̄) >= 0)) * np.exp(R_mat) * e_hat) / scale_2

    I2 = -1 * ξₚ * np.log(scale_2)

    def J2_without_e_fnc(x):
        return xi_d * np.exp(R_mat) * q2_tilde_fnc(x) * (γ1 * x + γ2 * F_mat * x ** 2 + γ2_plus * x * (x * F_mat - F̄) ** (power - 1) * ((x * F_mat - F̄) >= 0)) * norm.pdf(x,β𝘧,np.sqrt(σᵦ))

    J2_without_e = quad_int(J2_without_e_fnc, a, b, n, 'legendre')
    J2_with_e = J2_without_e * e_hat

    R2 = (I2 - J2_with_e) / ξₚ
    π̃2 = (1 - weight) * np.exp(-1 / ξₚ * I2)
    π̃1_norm = π̃1 / (π̃1 + π̃2)
    π̃2_norm = 1 - π̃1_norm

    # step 4 (b) updating e based on first order conditions
    expec_e_sum = (π̃1_norm * J1_without_e + π̃2_norm * J2_without_e)

    B1 = v0_dr - v0_df * np.exp(R_mat) - expec_e_sum
    C1 = -δ * κ
    e = -C1 / B1
    e_star = e

    J1 = J1_without_e * e_star
    J2 = J2_without_e * e_star

    # Step (3) calculating implied entropies
    I_term = -1 * ξₚ * np.log(π̃1 + π̃2)

    R1 = (I1 - J1) / ξₚ
    R2 = (I2 - J2) / ξₚ
    
    # Step (5) solving for adjusted drift
    drift_distort = (π̃1_norm * J1 + π̃2_norm * J2)

    if weight == 0 or weight == 1:
        RE = π̃1_norm * R1 + π̃2_norm * R2
    else:
        RE = π̃1_norm * R1 + π̃2_norm * R2 + π̃1_norm * np.log(
            π̃1_norm / weight) + π̃2_norm * np.log(π̃2_norm / (1 - weight))

    RE_total = ξₚ * RE

    # Step (6) and (7) Formulating HJB False Transient ODE parameters, See remark 4 for more details
    A = -δ * np.ones(R_mat.shape)
    B_r = -e_star + ψ0 * (j ** ψ1) * np.exp(ψ1 * (K_mat - R_mat)) - 0.5 * (σ𝘳 ** 2)
    B_f = e_star * np.exp(R_mat)
    B_k = μ̄ₖ + ϕ0 * np.log(1 + i * ϕ1) - 0.5 * (σ𝘬 ** 2)
    C_rr = 0.5 * σ𝘳 ** 2 * np.ones(R_mat.shape)
    C_ff = np.zeros(R_mat.shape)
    C_kk = 0.5 * σ𝘬 ** 2 * np.ones(R_mat.shape)
    D = δ * κ * np.log(e_star) + δ * κ * R_mat + δ * (1 - κ) * (np.log(α - i - j) + K_mat) + drift_distort + RE_total # + I_term 

    # Step (8) solving linear system using a conjugate-gradient method in C++, see remark 5, 6 for more details
    out = PDESolver(stateSpace, A, B_r, B_f, B_k, C_rr, C_ff, C_kk, D, v0, ε, 'False Transient')

    out_comp = out[2].reshape(v0.shape,order = "F")
    
    # Calculating PDE Error and False Transient (lhs) Error
    PDE_rhs = A * v0 + B_r * v0_dr + B_f * v0_df + B_k * v0_dk + C_rr * v0_drr + C_kk * v0_dkk + C_ff * v0_dff + D
    PDE_Err = np.max(abs(PDE_rhs))
    FC_Err = np.max(abs((out_comp - v0)))
    if episode % 100 == 0:
        print("Episode {:d}: PDE Error: {:.10f}; False Transient Error: {:.10f}; Iterations: {:d}; CG Error: {:.10f}" .format(episode, PDE_Err, FC_Err, out[0], out[1]))
    episode += 1
    
    # step 9: keep iterating until convergence
    v0 = out_comp

print("Episode {:d}: PDE Error: {:.10f}; False Transient Error: {:.10f}; Iterations: {:d}; CG Error: {:.10f}" .format(episode, PDE_Err, FC_Err, out[0], out[1]))
print("--- %s seconds ---" % (time.time() - start_time))


Cobweb Passed, iterations: 1, i error:   0.000000, j error:   0.000000
Episode 1: PDE Error: 0.0000062475; False Transient Error: 0.0000000100; Iterations: 2; CG Error: 0.0000000001
Episode 2: PDE Error: 0.0000062475; False Transient Error: 0.0000000100; Iterations: 2; CG Error: 0.0000000001
--- 24.203909158706665 seconds ---


#### Remark 1
The choices of ${\eta}$ in step (4) and $\epsilon$ in step (7) are made by trading off increases in speed of convergence, achieved by increasing their magnitudes, and enhancing stability of the iterative algorithm, achieved by decreasing their magnitudes.


#### Remark 2
We discretize the state space of $x$ using a set number of points along each of the three dimensions and impose a  fixed step size between points for each of these dimensions.  For interior points, we approximate the first derivatives using a first-order upwind scheme while the second derivatives are calculated using a central difference scheme. Upwind schemes are one-sided difference approximations that use the sign of the drifts for the states to determine the direction of the difference.  (See, for instance,  "An Introduction to Finite Difference Methods for PDE methods in Finance" by Agnes Tourin, Fields Institute.)  At boundary points we sometimes only have one option used in the approximation. We use a symmetric second difference approximation whenever possible and switch to a one-sided approximation as needed at boundary points.  With this construction, we have reduced the right-hand side of equaton in step (7) as a matrix applied to the value function at the chosen set of discrete points.

#### Remark 3

The term of interest is:

\begin{eqnarray}
(\kappa -1) \left[ \gamma_1 \beta + \gamma_2 \beta^2 f + \gamma_3\beta\left(\beta f - {\overline \gamma} \right)^2 \mathbb{1}(\beta f > {\overline \gamma} ) \right] \exp(r) e
\end{eqnarray}

Take ${\widehat e}$ from the previous iteration.


##### Low damage model

Since  $\gamma_3 = 0$ for the low damage model, use quasi analytical approach.

We wish to compute:
\begin{eqnarray}
{\mathcal I}_1  = - \xi_a \log E\left( \exp\left[ - {\frac 1 {\xi_a}} \left(  {\sf a}_1 + {\sf b}_1 \beta  + {\frac {{\sf c}_1} 2}   \beta ^2 \right) \right] \mid i \right)  .
\end{eqnarray}
where
\begin{align*}
{\sf a}_1& = 0,  \cr
 {\sf b}_1   & =  (\kappa - 1) {\widehat e} \exp(r)  \gamma_1, \cr
{\sf c}_1   & =  (\kappa -1) {\widehat e} \exp(r)  f \gamma_2.
\end{align*}





We apply a well known __complete-the-sqaures__ exploiting the exponential quadratic form of the normal density function.
Specifically, we compute the worst-case precision by equating the log- quadratic terms in $\beta$
\begin{eqnarray}
 \lambda  + {\frac {{\sf c}_1}{\xi_a}} = {\widetilde \lambda}_1
\end{eqnarray}
where $\lambda$ is the baseline precision and ${\widetilde \lambda}_i$ is the worst-case precision.

For calculating the worst-case mean, we equate first order terms in $\beta$
\begin{eqnarray}
{\overline \beta} \lambda - {\frac 1 {\xi_a}}  {\sf b}_1 = {\widetilde \beta}_1 {\widetilde \lambda}_1
\end{eqnarray}
Thus the worst-case mean is:
\begin{eqnarray}
{\widetilde \beta}_1   = {\overline \beta}\left({\frac {\lambda}{{\widetilde \lambda}_1}} \right)   - {\frac 1 {\xi_a {{\widetilde \lambda}_1 }}} {\sf b}_1  = {\overline \beta} -  {\frac {{\sf c}_1}{\xi_a \widetilde \lambda_i}}{\overline \beta}
-  {\frac 1 {\xi_a {{\widetilde \lambda}_1 }}} {\sf b}_1
\end{eqnarray}
Finally, we compute ${\mathcal I}_i$ by a) bringing it inside an exponential term,  b) multiplying this by the worst-case normal density that we just deduced, and c) equating the constant terms:
\begin{equation*}
- {\frac 1 {\xi_a}} {\mathcal I}_1   +   {\frac 1 2} \log {\widetilde \lambda}_1 - {\frac 1   2} \left({\widetilde  \beta}_1 \right)^2 {\widetilde \lambda}_1  = - {\frac 1 {\xi_a}} {\sf a}_1 + {\frac 1  2} \log \lambda - {\frac 1 2} \left({\overline \beta} \right)^2 \lambda .
\end{equation*}

Thus
\begin{equation*}
{\mathcal I}_1  = {\sf a}_1  -   {\frac {\xi_a}  2} \log \lambda + {\frac {\xi_a}  2} \log {\widetilde \lambda}_1
 + {\frac {\xi_a \lambda}  2} \left({\overline \beta} \right)^2   - {\frac {\xi_a {\widetilde \lambda}_1}  2} \left({\widetilde  \beta}_1 \right)^2
\end{equation*}


\begin{eqnarray}
{\mathcal R}_1  = {\frac 1 {\xi_a}} \left({\mathcal I}_1 -  \left[ {\sf a}_1  + {\sf b}_1 {\widetilde \beta}_1
+ {\sf c}_1  \left({\widetilde \beta}_1 \right)^2 + {\frac {{\sf c}_1} { {\widetilde \lambda}_1 }} \right]  \right)
\end{eqnarray}

\begin{eqnarray}
{\mathcal J}_1(e) = (\kappa -1)  \left( \gamma_1 {\widetilde \beta}_1 + \gamma_2 \left[  \left({\widetilde \beta}_1\right)^2
+ {\frac 1 {\widetilde \lambda}_1}\right]
f\right)  \exp(r) e
\end{eqnarray}

##### High damage model

We must proceed differently for the high damage model with numerical computation.
Form
\begin{eqnarray}
{\rm num}_2(\beta)  = \exp\left( - {\frac 1 \xi_a} (\kappa -1) \left[ \gamma_1 \beta + \gamma_2 \beta^2 f + \gamma_2^+ \beta\left(\beta f - {\overline \gamma} \right) \mathbb{1}(\beta f > {\overline \gamma} ) \right] \exp(r) {\widehat e}\right),
\end{eqnarray}
compute:
\begin{eqnarray}
{\rm scale}_2 = \int_\beta {\rm num}_2(\beta) p(\beta) d\beta
\end{eqnarray}

via Gauss-Hermite quadrature, 
and form a density conditioned on model two:
\begin{eqnarray}
{\widetilde q}_2(\beta)  = {\frac {{\rm num}(\beta)}{{\rm scale}2}}
\end{eqnarray}
relative to $p(\beta)$.  


Also compute:
\begin{eqnarray}
{\mathcal  I}_2 =  -\xi_a \log {\rm scale}_2,
\end{eqnarray}
along with:
\begin{eqnarray}
{\mathcal J}_2(e) = {\frac {(\kappa - 1) \left( \displaystyle\int_\beta    \left[ \gamma_1 \beta + \gamma_2 \beta^2 f + \gamma_2^+\beta\left(\beta f - {\overline \gamma} \right) \mathbb{1}(\beta f > {\overline \gamma} ) \right] 
{\rm num}_2(\beta)p(\beta) d \beta \right) \exp(r) e}{{\rm scale}_2}}
\end{eqnarray}
using Gaussian-Hermite quadrature for the numerator integral. 
Notice that ${\mathcal J}_2(e)$ can be computed for an arbitrary $e$ since the numerator integral does not depend on $e$.

With these computations in hand, form relative entropy conditioned on model two:
\begin{eqnarray}
{\mathcal R}_2 = {\frac 1 {\xi_a}} \left[{\mathcal I}_2 - {\mathcal J}_2\left( {\widehat e} \right) \right] .
\end{eqnarray}
Notice that ${\mathcal J}_2(e)$ can be computed for an arbitrary $e$ since the numerator integral does not depend on $e$.

##### Distorted model Probabilities


#### Remark 4
Recall that in order to solve the prefernce HJB model, we transform the question in solving a linear system of equations.
\begin{eqnarray}
{\frac {V(x) - {\widetilde V}(x)} \epsilon}  & =  &   V(x) \cdot A(x)+{\frac {\partial V}{\partial x}} (x) \cdot  B(x)   + {\frac {\partial^2 V}{\partial x \partial x'}}(x) \cdot C(x)+ D(x) 
\end{eqnarray}
Where
\begin{eqnarray}
A(x) &=& - \delta \\
B(x) &=& {\widehat \mu}(x)\\
C(x) &=& {\frac 1 2} \textrm{trace} \left[\sigma_X(x)' I \sigma_X(x) \right] \\
D(x) &=& \delta (1 - \kappa) \left( \log \left[ {\alpha}  - {\widehat i}(x)  - {\widehat j} (x)  \right] + k -  d  \right) + \delta \kappa \left[ \log {\widehat e}(x)   +  r \right] + {\frac {\xi_m} 2} {\widehat g}(x) \cdot {\widehat g}(x)   + \xi_p {\widehat {\mathbb I}}(x) \\
\end{eqnarray}

Precisely speaking our value function is a three-dimension tensor $ V(R,T,K) $ with default grid size of (181,161,121) in "F" order. In order to solve the linear system, we reshape $ V(x) $ into a one dimension array with size of (3526061,1) and conduct similar adjustments to $ A(x) $, $ B(x) $, $ C(x) $ and $ D(x) $ respectively.

In "F" order, when we call our initial value function at grid point (i,j,k) $ V(i,j,k) $, we instead call equivalent reshaped value fnction $V(i + j * N_i + k * N_i * N_j)$ where $N_i$, $N_j$ and $N_k$ stands for number of grid poitns in that dimension.



#### Remark 5

We solve the matrix counterpart to the equation in step (8) using the conjugate gradient algorithm.  This is a well known iterative algorithm designed to  solve a minimization problem:  $\frac 1 2 (\Lambda y - \lambda)'(\Lambda y - \lambda)$ for a  nonsingular matrix $\Lambda$ and vector $\lambda$.   The $y$ that minimizes this expression  satisfies the linear equation $\Lambda y = \lambda$.  The matrix $\Lambda$ and vector $\lambda$ come from the numerical approximation of below equation.  We measure the conjugate gradient error by

\begin{equation}
\sqrt{{\frac {(\Lambda y - \lambda)'(\Lambda y - \lambda)}{\lambda'\lambda}}}.
\end{equation}

We prespecify a conjugate gradient error bound and a bound on the difference in value functions between iterations and take as the starting point for conjugate gradient the output from the previous iteration.  We achieve convergence when the difference in value functions between iterations satisfies a prespecified error bound.  Upon convergence, we compute the maximum error for the matrix approximation to the right-hand side of equation system in step (7). We call this the maximum pde error.

#### Remark 6
While we are computing one-sided difference approximations at boundary points, we are not imposing additional boundary conditions on our finite state space as is often done when solving pde's with regular boundaries. Instead we aim to approximate pde solutions for the stochastic differential equation with unattainable boundaries.


#### Remark 7

In choosing the tolerance level, we tested the time it takes to solve the PDE numerically is decreasing with the size of $\epsilon$, while the stability of the program is also decreasing with $\epsilon$. 
\begin{eqnarray*}
\max_x \frac{|V(x)-\widetilde{V}(x)|}{\epsilon} < tol
\end{eqnarray*}

We choose time varying $\epsilon$ to balance the efficiency of the program and for room to explore different magnitude of the uncertainty.
* We used variable step sizes to speed program up while ensuring convergence: 0.3 for the first 1000 iterations, 0.2 for 1001-2000 iterations and 0.1 going forward.
*  For averse low damage case, normal variable step sizes scheme didn't work so we used 0.05 after 7000 iterations. 
*  We used fixed step size at 0.1 for growth neutral case to ensure convergence

Below table listed the convergence criteria and final errors for solving HJBs at different preference setting. We solved all models listed in this table by False Transient Algorithm. All outer loop convergence were evaluted on the change in value functions. 

| ambiguity    |    damage    | Cobweb tolerance |  outer loop tolerance  | CG tolerance |  PDE Error |
|:-:|:-:|:-:|:-:|:-:|:-:|
averse|  high  | 1e-5 | 1e-8 | 1e-10 | 6.88e-6 |
averse| weighted | 1e-5 | 1e-8 | 1e-10 | 6.25e-6|
averse|low| 1e-5 | 1e-8 | 1e-10 | 3.83e-6|
neutral|high  | 1e-5 | 1e-8 | 1e-10 | 5.67e-6|
neutral|weighted  | 1e-5 | 1e-8 | 1e-10 | 4.60e-6|
neutral|low  | 1e-5 | 1e-8 | 1e-10 | 3.73e-6|

Below table listed the convergence criteria and final errors for solving HJBs under growth settings. 


| ambiguity    |    damage    | Cobweb tolerance |  outer loop tolerance  | CG tolerance |  PDE Error |
|:-:|:-:|:-:|:-:|:-:|:-:|
averse | -  | 1e-5 | 1e-8 | 1e-10 | 1.59e-5
neutral  | -  | 1e-5 | 1e-8 | 1e-10 | 1.27e-5

# Simulation

In [ ]:
class GridInterp():

    def __init__(self, grids, values, method = 'Linear'):

        # unpacking
        self.grids = grids
        (self.xs, self.ys, self.zs) = grids
        self.nx = len(self.xs)
        self.ny = len(self.ys)
        self.nz = len(self.zs)
        
        self.values = values

        assert (self.nx, self.ny, self.nz) == values.shape, "ValueError: Dimensions not match"
        self.method = method

    def get_value(self, x, y, z):

        if self.method == 'Linear':
            
            func = RegularGridInterpolator(self.grids, self.values)
            return func([x,y,z])[0]

        elif self.method == 'Spline':

            func1 = CubicSpline(self.xs, self.values)
            yzSpace = func1(x)
            
            func2 = CubicSpline(self.ys, yzSpace)
            zSpace = func2(y)
            
            func3 = CubicSpline(self.zs, zSpace)
            return func3(z)

        else:
            raise ValueError('Method Not Supported')

In [ ]:
method = 'Linear'
T = 100
pers = 4 * T
dt = T / pers
nDims = 5
its = 1

gridpoints = (R, F, K)

e_func_r = GridInterp(gridpoints, e, method)
def e_func(x):
    return e_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

j_func_r = GridInterp(gridpoints, j, method)
def j_func(x):
    return max(j_func_r.get_value(np.log(x[0]), x[2], np.log(x[1])), 0)

i_func_r = GridInterp(gridpoints, i, method)
def i_func(x):
    return i_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

v_drfunc_r = GridInterp(gridpoints, v0_dr, method)
def v_drfunc(x):
    return v_drfunc_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

v_dtfunc_r = GridInterp(gridpoints, v0_df, method)
def v_dtfunc(x):
    return v_dtfunc_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

v_dkfunc_r = GridInterp(gridpoints, v0_dk, method)
def v_dkfunc(x):
    return v_dkfunc_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

v_func_r = GridInterp(gridpoints, v0, method)
def v_func(x):
    return v_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

pi_tilde_1_func_r = GridInterp(gridpoints, π̃1 / (π̃1 + π̃2), method)
def pi_tilde_1_func(x):
    return pi_tilde_1_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

pi_tilde_2_func_r = GridInterp(gridpoints, π̃2 / (π̃1 + π̃2), method)
def pi_tilde_2_func(x):
    return pi_tilde_2_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

def scale_2_fnc(x):
    return np.exp(-1 / ξₚ * xi_d * (γ1 * x + γ2 * x ** 2 * F_mat + γ2_plus * x * (x * F_mat - F̄) ** (power - 1) * ((x * F_mat - F̄) >= 0)) * np.exp(R_mat) * e_hat)  * norm.pdf(x,β𝘧,np.sqrt(σᵦ))

scale_2 = quad_int(scale_2_fnc, a, b, n, 'legendre')

def q2_tilde_fnc(x):
    return np.exp(-1 / ξₚ * xi_d * (γ1 * x + γ2 * x ** 2 * F_mat + γ2_plus * x * (x * F_mat - F̄) ** (power - 1) * ((x * F_mat - F̄) >= 0)) * np.exp(R_mat) * e_hat) / scale_2

def base_model_drift_func(x):
    return np.exp(R_mat) * e * (γ1 * x + γ2 * x ** 2 * F_mat + γ̄2_plus * x * (x * F_mat - F̄) ** (power - 1) * ((x * F_mat - F̄) >= 0)) * norm.pdf(x,β𝘧,np.sqrt(σᵦ))
base_model_drift =  quad_int(base_model_drift_func, a, b, n, 'legendre')

mean_nordhaus = β̃1
lambda_tilde_nordhaus = λ̃1
nordhaus_model_drift = (γ1 * mean_nordhaus + γ2 * (1 / lambda_tilde_nordhaus + mean_nordhaus ** 2) * F_mat) * np.exp(R_mat) * e

def weitzman_model_drift_func(x):
    return np.exp(R_mat) * e * q2_tilde_fnc(x) * (γ1 * x + γ2 * x ** 2 * F_mat + γ2_plus * x * (x * F_mat - F̄ ) ** (power - 1) * ((x * F_mat - F̄) >= 0)) * norm.pdf(x,β𝘧,np.sqrt(σᵦ))
weitzman_model_drift = quad_int(weitzman_model_drift_func, a, b, n, 'legendre')

nordhaus_drift_func_r = GridInterp(gridpoints, nordhaus_model_drift, method)
def nordhaus_drift_func(x):
    return nordhaus_drift_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

weitzman_drift_func_r = GridInterp(gridpoints, weitzman_model_drift, method)
def weitzman_drift_func(x):
    return weitzman_drift_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

base_drift_func_r = GridInterp(gridpoints, base_model_drift, method)
def base_drift_func (x): 
    return base_drift_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

# function handles
def muR(x):
    return -e_func(x) + ψ0 * (j_func(x) * x[1] / x[0]) ** ψ1
def muK(x): 
    return (μ̄k + ϕ0 * np.log(1 + i_func(x) * ϕ1))
def muF(x):
    return e_func(x) * x[0]
def muD_base(x):
    return base_drift_func(x)
def muD_tilted(x):
    return pi_tilde_1_func(x) * nordhaus_drift_func(x) + (1 - pi_tilde_1_func(x)) * weitzman_drift_func(x)

def sigmaR(x):
    return np.zeros(x[:5].shape)
def sigmaK(x):
    return np.zeros(x[:5].shape)
def sigmaF(x):
    return np.zeros(x[:5].shape)
def sigmaD(x):
    return np.zeros(x[:5].shape)

# initial points
R_0 = 650
K_0 = 80 / α
F_0 = 870 - 580
initial_val = np.array([R_0, K_0, F_0])
D_0_base = muD_base(initial_val)
D_0_tilted = muD_tilted(initial_val)

# Set bounds
R_max_sim = np.exp(max(R))
K_max_sim = np.exp(max(K))
F_max_sim = max(F)
D_max_sim = 5.0

R_min_sim = np.exp(min(R))
K_min_sim = np.exp(min(K))
F_min_sim = min(F)
D_min_sim = -5

upperbounds = np.array([R_max_sim, K_max_sim, F_max_sim, D_max_sim, D_max_sim])
lowerbounds = np.array([R_min_sim, K_min_sim, F_min_sim, D_min_sim, D_min_sim])

hists = np.zeros([pers, nDims, its])
# hists = hists.copy()
e_hists = np.zeros([pers,its])
# e_hists = e_hists.copy()
j_hists = np.zeros([pers,its])
# j_hists = j_hists.copy()
i_hists = np.zeros([pers,its])
# i_hists = i_hists.copy()

v_dr_hists = np.zeros([pers,its])
v_dt_hists = np.zeros([pers,its])
v_dk_hists = np.zeros([pers,its])
v_hists = np.zeros([pers,its])

for iters in range(0,its):
    hist = np.zeros([pers,nDims])
    e_hist = np.zeros([pers,1])
    i_hist = np.zeros([pers,1])
    j_hist = np.zeros([pers,1])

    v_dr_hist = np.zeros([pers,1])
    v_dt_hist = np.zeros([pers,1])
    v_dk_hist = np.zeros([pers,1])
    v_hist = np.zeros([pers,1])

    hist[0,:] = [R_0, K_0, F_0, D_0_base, D_0_tilted]
    e_hist[0] = e_func(hist[0,:]) * hist[0,0]
    i_hist[0] = i_func(hist[0,:]) * hist[0,1]
    j_hist[0] = j_func(hist[0,:]) * hist[0,0]
    v_dr_hist[0] = v_drfunc(hist[0,:])
    v_dt_hist[0] = v_dtfunc(hist[0,:])
    v_dk_hist[0] = v_dkfunc(hist[0,:])
    v_hist[0] = v_func(hist[0,:])

    for tm in range(1,pers):
        shock = norm.rvs(0,np.sqrt(dt),nDims)
        # print(muR(hist[tm-1,:]))
        hist[tm,0] = cap(hist[tm-1,0] * np.exp((muR(hist[tm-1,:])- 0.5 * sum((sigmaR(hist[tm-1,:])) ** 2))* dt + sigmaR(hist[tm-1,:]).dot(shock)),lowerbounds[0], upperbounds[0])
        hist[tm,1] = cap(hist[tm-1,1] * np.exp((muK(hist[tm-1,:])- 0.5 * sum((sigmaK(hist[tm-1,:])) ** 2))* dt + sigmaK(hist[tm-1,:]).dot(shock)),lowerbounds[1], upperbounds[1])
        hist[tm,2] = cap(hist[tm-1,2] + muF(hist[tm-1,:]) * dt + sigmaF(hist[tm-1,:]).dot(shock), lowerbounds[2], upperbounds[2])
        hist[tm,3] = cap(hist[tm-1,3] + muD_base(hist[tm-1,:]) * dt + sigmaD(hist[tm-1,:]).dot(shock), lowerbounds[3], upperbounds[3])
        hist[tm,4] = cap(hist[tm-1,4] + muD_tilted(hist[tm-1,:]) * dt + sigmaD(hist[tm-1,:]).dot(shock), lowerbounds[4], upperbounds[4])

        e_hist[tm] = e_func(hist[tm-1,:]) * hist[tm-1,0]
        i_hist[tm] = i_func(hist[tm-1,:]) * hist[tm-1,1]
        j_hist[tm] = j_func(hist[tm-1,:]) * hist[tm-1,0]

        v_dr_hist[tm] = v_drfunc(hist[tm-1,:])
        v_dt_hist[tm] = v_dtfunc(hist[tm-1,:])
        v_dk_hist[tm] = v_dkfunc(hist[tm-1,:])
        v_hist[tm] = v_func(hist[tm-1,:])

    hists[:,:,iters] = hist
    e_hists[:,[iters]] = e_hist
    i_hists[:,[iters]] = i_hist
    j_hists[:,[iters]] = j_hist

    v_dr_hists[:,[iters]] = v_dr_hist
    v_dt_hists[:,[iters]] = v_dt_hist
    v_dk_hists[:,[iters]] = v_dk_hist
    v_hists[:,[iters]] = v_hist

## SCC Calculation Feyman Kac

In [ ]:
# Loading smart guesses
base_ = loadmat(r'C:\Users\jiamingwang\Dropbox\share with John\Final Code\Preference\SCC_solution\SCC_base_averse_weighted.mat')
base_guess = base_['v0']
worst_ = loadmat(r'C:\Users\jiamingwang\Dropbox\share with John\Final Code\Preference\SCC_solution\SCC_worst_averse_weighted.mat')
worst_guess = worst_['v0']


As in the body of the paper, consider impulse response functions for
the logarithm of damages in the future induced by a marginal change in emissions today.  The responses are  necessarily nonlinear impulse responses and hence will be state-dependent.  The marginal  emissions change induces
an  impact on $\log D_{t+u}$ given by

$$\begin{eqnarray}
\left([\nabla \Gamma](\beta F_t) \beta  + \zeta_D(Z_t) \cdot \begin{bmatrix} 1 \cr 0 \end{bmatrix} \right)
+ \int_0^u [\nabla^2 \Gamma](\beta F_{t+\tau}) \beta^2 E_{t+\tau} d \tau.
\end{eqnarray}$$

The first contribution occurs on impact, and the second one accumulates through the effect of current emissions on the state variable $f$.

Consider the specification where damages enter the utility function discounted and multiplied by $\delta (1-\kappa)$.  Recall that by doing  some simple accounting and exploiting the exponential discounting used for the discounted marginal damage response,   we ecombine all the date $\tau$ contributions for $u \ge \tau$ to obtain

\begin{eqnarray}
\exp(-\delta \tau)  (1-\kappa) [\nabla^2 \Gamma]( \beta F_{t+\tau} ) \beta^2 E_{t+\tau},
\end{eqnarray}

along with the initial term

$$\begin{eqnarray}
\exp(-\delta \tau) \delta (1-\kappa) \left([\nabla \Gamma](\beta F_t) \beta  + \zeta_D(Z_t) \cdot \begin{bmatrix} 1 \cr 0 \end{bmatrix} \right).
\end{eqnarray}$$

The external part to the social cost of carbon is the expected exponentially discounted future impulse responses.   In the absence of ambiguity and robustness concerns it is given by

$$\begin{align} 
& \delta (1-\kappa) \left([\nabla \Gamma](\beta F_t) \beta  + \zeta_D(Z_t) \cdot \begin{bmatrix} 1 \cr 0 \end{bmatrix} \right) \int_0^\infty
\exp(-\delta \tau) d\tau  \cr
& + E \left[ \int_0^\infty \exp(-\delta \tau) (1-\kappa) [\nabla^2 \Gamma]( \beta F_{t+\tau} ) \beta^2 E_{t+\tau} d \tau \mid X_t = x \right]
\tag{1}
\end{align}$$

divided by the date $t$ marginal utility of consumption. 

By integrating the exponential function in the first expression, the $\delta$ drops out resulting in

\begin{eqnarray}
(1-\kappa) \left([\nabla \Gamma](\beta F_t) \beta  + \zeta_D(Z_t) \cdot \begin{bmatrix} 1 \cr 0 \end{bmatrix} \right),
\end{eqnarray}

which is one of the two terms in formula $(1)$ for $ecc$.


Since the second term is a  discounted expected value, it solves a so-called __Feynman-Kac (FK) equation__.   Formally, we are interested in the solution $\Phi$ to the forward-looking equation

$$\begin{align} \label{FKsolution}
\Phi(X_t) & = {\mathbb E} \left[ \int_0^\infty \exp( - \delta \tau) \Psi(X_{t+\tau}) d \tau \mid X_t \right] \cr
& = \exp(\delta t)  \int_t^\infty \exp( - \delta \tau) {\mathbb E} \left[ \Psi(X_{\tau}) \mid X_t \right] d \tau
\tag{2}
\end{align}$$

for a pre-specified $\Psi$. Specifically, let

\begin{eqnarray}
\Psi(x) =   (1-\kappa) [\nabla^2 \Gamma] ( \beta f ) \beta^2  e^*(x) \exp(r).
\end{eqnarray}

To provide a heuristic reminder of form and rationale for the FK equation,  we obtain the drift of the process $\{ \Phi(X_t) : t \ge 0 \}$ of the left-hand hand side of formula $(2)$ via Ito's formula for $X_t = x$ as

\begin{eqnarray}
{\frac {\partial \Phi}{\partial x}} (x) \cdot \mu_X[x,a^*(x)]
+{\frac 1 2} \textrm{trace} \left[\sigma_X(x)' {\frac {\partial^2 \Phi}{\partial x \partial x'}}(x) \sigma_X(x) \right],
\end{eqnarray}

where $a^*$ is the maximizing decision rule. Differentiating the right-hand side of $(2)$ with respect to $t$ gives an alternative formula for this drift:

\begin{eqnarray}
\delta \Phi(x)  - \Psi(x).
\end{eqnarray}

By equating these, we obtain the FK or (more generally) resolvent equation:

\begin{equation} 
- \delta \Phi(x)  + {\frac {\partial \Phi}{\partial x}} (x) \cdot \mu_X[x,a^*(x)]
+{\frac 1 2} \textrm{trace} \left[\sigma_X(x)' {\frac {\partial^2 \Phi}{\partial x \partial x'}}(x) \sigma_X(x) \right] + \Psi(x)=0.
\tag{3}
\end{equation}


By differentiating the HJB equation with respect to $f$ and applying the __Envelope Theorem__, it can be shown that the solution $\Phi$  to the FK equation satisfies


\begin{eqnarray}
\Phi(x) = - V_f(x).
\end{eqnarray}


The analogous arguments apply in the presence of ambiguity and robustness concerns except that we use the altered probability distribution 
when computing expectations.  

For the uncertainty aversion we need to address the tilted probability $Q^*$. We solve numerically a corresponding Feynman-Kac equation with the worst-case evolution.  We let
\begin{align*}
\widetilde{\Psi(x)} =  (1 - \kappa) \int_\Theta \nabla^2 \Gamma(\beta f) \beta^2  dQ^*\left(\theta \mid x \right)  {\hat e \left(x  \right) \exp(r)} .
\end{align*}
The corresponding $\widetilde{\Phi}$ is to the solution of the below PDE
\begin{align*}
-\delta \widetilde{\Phi(x)} + {\frac {\partial \widetilde{\Phi}}{\partial x}} (x) \cdot \mu_X[x,a^*(x)] +{\frac 1 2} \textrm{trace} \left[\sigma_X(x)' {\frac {\partial^2 \widetilde{\Phi}}{\partial x \partial x'}}(x) \sigma_X(x) \right] + \widetilde{\Psi(x)}=0.
\end{align*}

Note that this PDE is different from the above one in only the last term $\Psi$. The previous case is a special case for the worst-case evolution, with the uncertainty parameter $\xi_a=\infty$.

In calculating $\widetilde{\Psi}$ we choose "Gauss-Legendre" quadrature with 30 integration points on $[\beta_f-5\sigma_f,\beta_f+5\sigma_f] $ in computing the term that involves high damage models.

We again solve the PDE using the conjugate gradient solver using the below form:
\begin{eqnarray}
\widetilde{\Phi(x)} \cdot A(x)+{\frac {\partial\widetilde{ \Phi}}{\partial x}} (x) \cdot  B(x)   + trace [S(x)'\frac {\partial^2 \widetilde{\Phi}}{\partial x \partial x'}(x)S(x)] + D(x) & =  &  0
\end{eqnarray}
With the following coefficients:

\begin{eqnarray}
A(x) &=& - \delta \\
B(x) &=& \mu_X[x,a^*(x)]\\
C(x) &=& {\frac 1 2} \sigma_X(x)'I\sigma_X(x)  \\
D(x) &=& \widetilde{\Psi(x)}\\
\end{eqnarray}
with $C(x) = S(x)'S(x)$

In [ ]:
if ξₚ > 100:  # We consider for ξₚ level over 100 as 
    
    # Under ambiguity neutrality, we can write explicit form solution for Social Cost of Carbon
    MC = δ * (1-κ) / (α * np.exp(K_mat) - i * np.exp(K_mat) - j * np.exp(R_mat))      # Marginal utility of consumption
    ME = δ * κ / (e * np.exp(R_mat))      # Marginal utility of emission
    SCC = 1000 * ME / MC
    SCC_func_r = GridInterp(gridpoints, SCC, method)
    
    def SCC_func(x): 
        return SCC_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))
    
    SCC_values = np.zeros([pers,its])
    
    # Interpolating values from simulated trajectory
    for tm in range(pers):
        for path in range(its): 
            SCC_values[tm, path] = SCC_func(hists[tm,:,path])

    SCC_total = np.mean(SCC_values,axis = 1)

    SCCs['SCC'] = SCC_total
    
else:

    # Base model
    def base_model_flow_func(x):
        return (γ2 * x ** 2 + γ̄2_plus * x ** 2 * ((x * F_mat - F̄) >=0)) * np.exp(R_mat) * e *  norm.pdf(x,β𝘧,np.sqrt(σᵦ))
    base_model_flow = quad_int(base_model_flow_func, a, b, n, 'legendre')
    flow_base = base_model_flow

    # input for solver

    A = -δ * np.ones(R_mat.shape)
    ####
    B_r = -e + ψ0 * (j ** ψ1) * np.exp(ψ1 * (K_mat - R_mat)) - 0.5 * (σ𝘳 ** 2)
    B_k = μ̄ₖ + ϕ0 * np.log(1 + i * ϕ1) - 0.5 * (σ𝘬 ** 2)
    B_f = e * np.exp(R_mat)
    C_rr = 0.5 * σ𝘳 ** 2 * np.ones(R_mat.shape)
    C_kk = 0.5 * σ𝘬 ** 2 * np.ones(R_mat.shape)
    C_ff = np.zeros(R_mat.shape)
    D = flow_base

    out = PDESolver(stateSpace, A, B_r, B_f, B_k, C_rr, C_ff, C_kk, D, base_guess, solverType='Feyman Kac')
    v0_base = out[2].reshape(v0.shape, order="F")
    v0_base = v0_base

    v0_dr_base = finiteDiff(v0_base,0,1,hR) 
    v0_df_base = finiteDiff(v0_base,1,1,hF)
    v0_dk_base = finiteDiff(v0_base,2,1,hK)

    v0_drr_base = finiteDiff(v0_base,0,2,hR)
    v0_dff_base = finiteDiff(v0_base,1,2,hF)
    v0_dkk_base = finiteDiff(v0_base,2,2,hK)

    v0_drr_base[v0_dr_base < 1e-16] = 0
    v0_dr_base[v0_dr_base < 1e-16] = 1e-16

    PDE_rhs = A * v0_base + B_r * v0_dr_base + B_f * v0_df_base + B_k * v0_dk_base + C_rr * v0_drr_base + C_kk * v0_dkk_base + C_ff * v0_dff_base + D
    PDE_Err = np.max(abs(PDE_rhs))
    print("Feyman Kac Base Model Solved. PDE Error: {:.10f}; Iterations: {:d}; CG Error: {:.10f}".format(PDE_Err, out[0], out[1]))

    # Worst Model
    mean_nordhaus = β̃1
    lambda_tilde_nordhaus = λ̃1
    def scale_2_fnc(x):
        return np.exp(-1 / ξₚ * xi_d * (γ1 * x + γ2 * x ** 2 * F_mat + γ2_plus * x * (x * F_mat - F̄) ** (power - 1) * ((x * F_mat - F̄) >= 0)) * np.exp(R_mat) * e)  * norm.pdf(x,β𝘧,np.sqrt(σᵦ))

    scale_2 = quad_int(scale_2_fnc, a, b, n, 'legendre')

    def q2_tilde_fnc(x):
        return np.exp(-1 / ξₚ * xi_d * (γ1 * x + γ2 * x ** 2 * F_mat + γ2_plus * x * (x * F_mat - F̄) ** (power - 1) * ((x * F_mat - F̄) >= 0)) * np.exp(R_mat) * e) / scale_2

    nordhaus_model_flow = (γ2 * (1 / lambda_tilde_nordhaus + mean_nordhaus ** 2)) * np.exp(R_mat) * e 
    # weitzman_model_flow_func = @(x) q2_tilde_1_fnc(x) .*(gamma_2.*x.^2 +gamma_2_plus.*x.^2.*((x.*t_mat-f_bar)>=0)).*exp(r_mat).*e .*normpdf(x,beta_f,sqrt(var_beta_f));
    def weitzman_model_flow_func(x): 
        return q2_tilde_fnc(x) * (γ2 * x ** 2 + γ2_plus * x ** 2 * ((x * F_mat - F̄) >= 0 )) * np.exp(R_mat) * e * norm.pdf(x,β𝘧,np.sqrt(σᵦ))
    weitzman_model_flow = quad_int(weitzman_model_flow_func, a, b, n, 'legendre')

    I1 = a1 - 0.5 * np.log(λ) * ξₚ + 0.5 * np.log(λ̃1) * ξₚ + 0.5 * λ * β𝘧 ** 2 * ξₚ - 0.5 * λ̃1 * (β̃1) ** 2 * ξₚ
    I2 = -1 * ξₚ * np.log(scale_2)
    π̃1 = (weight) * np.exp(-1 / ξₚ * I1)
    π̃2 = (1 - weight) * np.exp(-1 / ξₚ * I2)
    π̃1_norm = π̃1 / (π̃1 + π̃2)
    π̃2_norm = 1 - π̃1_norm

    flow_tilted = π̃1_norm * nordhaus_model_flow + π̃2_norm * weitzman_model_flow

    A = -δ * np.ones(R_mat.shape)
    ####
    B_r = -e + ψ0 * (j ** ψ1) * np.exp(ψ1 * (K_mat - R_mat)) - 0.5 * (σ𝘳 ** 2)
    B_k = μ̄ₖ + ϕ0 * np.log(1 + i * ϕ1) - 0.5 * (σ𝘬 ** 2)
    B_f = e * np.exp(R_mat)
    C_rr = 0.5 * σ𝘳 ** 2 * np.ones(R_mat.shape)
    C_kk = 0.5 * σ𝘬 ** 2 * np.ones(R_mat.shape)
    C_ff = np.zeros(R_mat.shape)
    D = flow_tilted

    out = PDESolver(stateSpace, A, B_r, B_f, B_k, C_rr, C_ff, C_kk, D, worst_guess, solverType='Feyman Kac')
    v0_worst = out[2].reshape(v0.shape, order="F")
    v0_worst = v0_worst

    v0_dr_worst = finiteDiff(v0_worst,0,1,hR) 
    v0_df_worst = finiteDiff(v0_worst,1,1,hF)
    v0_dk_worst = finiteDiff(v0_worst,2,1,hK)

    v0_drr_worst = finiteDiff(v0_worst,0,2,hR)
    v0_dff_worst = finiteDiff(v0_worst,1,2,hF)
    v0_dkk_worst = finiteDiff(v0_worst,2,2,hK)
    v0_drr_worst[v0_dr_worst < 1e-16] = 0
    v0_dr_worst[v0_dr_worst < 1e-16] = 1e-16

    PDE_rhs = A * v0_worst + B_r * v0_dr_worst + B_f * v0_df_worst + B_k * v0_dk_worst + C_rr * v0_drr_worst + C_kk * v0_dkk_worst + C_ff * v0_dff_worst + D
    PDE_Err = np.max(abs(PDE_rhs))
    print("Feyman Kac Worst Model Solved. PDE Error: {:.10f}; Iterations: {:d}; CG Error: {:.10f}".format(PDE_Err, out[0], out[1]))


    # SCC decomposition

    v0_dr = finiteDiff(v0,0,1,hR) 
    v0_df = finiteDiff(v0,1,1,hF)
    v0_dk = finiteDiff(v0,2,1,hK)

    v0_drr = finiteDiff(v0,0,2,hR)
    v0_dff = finiteDiff(v0,1,2,hF)
    v0_dkk = finiteDiff(v0,2,2,hK)

    v0_drr[v0_dr < 1e-16] = 0
    v0_dr[v0_dr < 1e-16] = 1e-16

    gridpoints = (R, F, K)  # can modify

    MC = δ * (1-κ) / (α * np.exp(K_mat) - i * np.exp(K_mat) - j * np.exp(R_mat))
    ME = δ * κ / (e * np.exp(R_mat))
    SCC = 1000 * ME / MC
    SCC_func_r = GridInterp(gridpoints, SCC, method)

    def SCC_func(x): 
        return SCC_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

    ME1 = v0_dr * np.exp(-R_mat)
    SCC1 = 1000 * ME1 / MC
    SCC1_func_r = GridInterp(gridpoints, SCC1, method)
    def SCC1_func(x):
        return SCC1_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

    ME2_base = (1-κ) * v0_base
    SCC2_base = 1000 * ME2_base / MC
    SCC2_base_func_r = GridInterp(gridpoints, SCC2_base, method)
    def SCC2_base_func(x):
        return SCC2_base_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

    def V_d_baseline_func(x):
        return xi_d * (γ1 * x + γ2 * F_mat * x** 2 +
                        γ̄2_plus * x * (x * F_mat - F̄) * (power - 1)
                        * ((x * F_mat - F̄) >= 0 )) * norm.pdf(x, β𝘧, np.sqrt(σᵦ))
    V_d_baseline = quad_int(V_d_baseline_func, a, b, n, 'legendre')
    ME2b = -V_d_baseline
    SCC2_V_d_baseline = 1000 * ME2b / MC
    SCC2_V_d_baseline_func_r = GridInterp(gridpoints, SCC2_V_d_baseline, method)
    def SCC2_V_d_baseline_func(x):
        return SCC2_V_d_baseline_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

    ME2_tilt = (1-κ) * v0_worst
    SCC2_tilt = 1000 * ME2_tilt / MC
    SCC2_tilt_func_r = GridInterp(gridpoints, SCC2_tilt, method)
    def SCC2_tilt_func(x):
        return SCC2_tilt_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))


    ME2b = -expec_e_sum * np.exp(-R_mat)
    SCC2_V_d_tilt_ = 1000 * ME2b / MC
    SCC2_V_d_tilt_func_r = GridInterp(gridpoints, SCC2_V_d_tilt_, method)
    def SCC2_V_d_tilt_func(x):
        return SCC2_V_d_tilt_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))


    SCC_values = np.zeros([pers,its])
    SCC1_values = np.zeros([pers,its])
    SCC2_base_values = np.zeros([pers,its])
    SCC2_tilt_values = np.zeros([pers,its])
    SCC2_V_d_baseline_values = np.zeros([pers,its])
    SCC2_V_d_tilt_values = np.zeros([pers,its])

    for tm in range(pers):
        for path in range(its):   # path is its?
            SCC_values[tm, path] = SCC_func(hists[tm,:,path])
            SCC1_values[tm, path] = SCC1_func(hists[tm,:,path])
            SCC2_base_values[tm, path] = SCC2_base_func(hists[tm,:,path]) 
            SCC2_tilt_values[tm, path] = SCC2_tilt_func(hists[tm,:,path])
            SCC2_V_d_baseline_values[tm, path] = SCC2_V_d_baseline_func(hists[tm,:,path])
            SCC2_V_d_tilt_values[tm, path] = SCC2_V_d_tilt_func(hists[tm,:,path])

    SCC_total = np.mean(SCC_values,axis = 1)     # Total SCC, 
    SCC_private = np.mean(SCC1_values,axis = 1)  # Private, resource scarcity
    SCC2_FK_base = np.mean(SCC2_base_values,axis = 1)  # ecc_bar, expectation part
    SCC2_FK_tilt = np.mean(SCC2_tilt_values,axis = 1)
    SCC2_V_d_baseline = np.mean(SCC2_V_d_baseline_values,axis = 1) # ecc_bar first part; instantaneous contribution
    SCC2_V_d_tilt = np.mean(SCC2_V_d_tilt_values,axis = 1)

    SCCs = {}
    SCCs['SCC'] = SCC_total
    SCCs['SCC1'] = SCC_private
    SCCs['SCC2'] = SCC2_FK_base + SCC2_V_d_baseline
    SCCs['SCC3'] = SCC2_V_d_tilt - SCC2_V_d_baseline + SCC2_FK_tilt - SCC2_FK_base

In [ ]:
SCCs['SCC3'][-1]

### Probabilities

In [ ]:
REs = {}
Dists = {}
a = β𝘧 - 5 * np.sqrt(σᵦ)
b = β𝘧 + 5 * np.sqrt(σᵦ)
a_10std = β𝘧 - 10 * np.sqrt(σᵦ)
b_10std = β𝘧 + 10 * np.sqrt(σᵦ)

RE_func_r = GridInterp(gridpoints, RE, method)
def RE_func(x):
    return RE_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

e_func_r = GridInterp(gridpoints, e, method)
def e_func(x):
    return e_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

pi_tilde_1_func_r = GridInterp(gridpoints, π̃1, method)
def pi_tilde_1_func(x):
    return pi_tilde_1_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

lambda_tilde_1_func_r = GridInterp(gridpoints, λ̃1, method)
def lambda_tilde_1_func(x):
    return lambda_tilde_1_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

beta_tilde_1_r = GridInterp(gridpoints, β̃1, method)
def beta_tilde_1_func(x):
    return beta_tilde_1_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

RE_plot = np.zeros(pers)
weight_plot = np.zeros(pers)
beta_f_space = np.linspace(a_10std,b_10std,200)

#Relative Entropy

if damageSpec == 'low':
    nordhaus_mean = np.zeros(pers)
    nordhaus_std = np.zeros(pers)

    for tm in range(pers):
        RE_plot[tm] = RE_func(hists[tm,:,0])
        weight_plot[tm] = pi_tilde_1_func(hists[tm,:,0])
        nordhaus_mean[tm] = beta_tilde_1_func(hists[tm,:,0])
        nordhaus_std[tm] = 1 / np.sqrt(lambda_tilde_1_func(hists[tm,:,0]))

    REs['RE'] = RE_plot
    REs['Weights'] = weight_plot
    REs['Shifted Mean'] = nordhaus_mean
    REs['Shifted Std'] = nordhaus_std

else:
    for tm in range(pers):
        RE_plot[tm] = RE_func(hists[tm,:,0])
        weight_plot[tm] = pi_tilde_1_func(hists[tm,:,0])

    REs['RE'] = RE_plot
    REs['Weights'] = weight_plot


original_dist = norm.pdf(beta_f_space, β𝘧, np.sqrt(σᵦ))
Dists['Original'] = original_dist
    
# probabilities (R,K,F)

for tm in [1,100,200,300,400]:
    R0 = hists[tm-1,0,0]
    K0 = hists[tm-1,1,0]
    F0 = hists[tm-1,2,0]

    # Weitzman
    def scale_2_fnc_prob(x):
        return np.exp(-1 / ξp * xi_d * (γ1 * x + γ2 * x ** 2 *  F0 + γ2_plus * x * (x * F0 - F̄) ** (power - 1) * ((x * F0 - F̄) >= 0)) * R0 * e_func([R0, K0, F0])) * norm.pdf(x, β𝘧, np.sqrt(σᵦ))
    scale_2_prob = quad_int(scale_2_fnc_prob, a, b, n, 'legendre')

    q2_tilde_fnc_prob = np.exp(-1 / ξp * xi_d * (γ1 * beta_f_space + γ2 * beta_f_space ** 2 * F0 + γ2_plus * beta_f_space * (beta_f_space * F0 - F̄) ** (power - 1) * ((beta_f_space * F0 - F̄) >= 0)) * R0* e_func([R0, K0, F0])) / scale_2_prob * norm.pdf(beta_f_space, β𝘧, np.sqrt(σᵦ))
    weitzman = q2_tilde_fnc_prob

    # Nordhaus
    mean_distort_nordhaus = beta_tilde_1_func([R0, K0, F0]) - β𝘧
    lambda_tilde_nordhaus = lambda_tilde_1_func([R0, K0, F0])
    nordhaus = norm.pdf(beta_f_space, mean_distort_nordhaus + β𝘧, 1 / np.sqrt(lambda_tilde_nordhaus))

    # weights
    Dists_weight = pi_tilde_1_func([R0, K0, F0])
    if damageSpec == 'High':
        Dists['Weitzman_year' + str(int((tm) / 4))] = weitzman
    elif damageSpec == 'Low':
        Dists['Nordhaus_year' + str(int((tm) / 4))] = nordhaus
    elif damageSpec == 'Weighted':
        Dists['Weitzman_year' + str(int((tm) / 4))] = weitzman
        Dists['Nordhaus_year' + str(int((tm) / 4))] = nordhaus
        Dists['Weighted_year' + str(int((tm) / 4))] = nordhaus * Dists_weight + weitzman * (1 - Dists_weight)


### Save data as smart guesses

In [20]:
base_ = loadmat(r'C:\Users\jiamingwang\Dropbox\share with John\Final Code/Growth_JY_Final/SCC_mat_Cumu_base_GrowthNoAmb_1e9.mat')
base_guess = base_['v0']
worst_ = loadmat(r'C:\Users\jiamingwang\Dropbox\share with John\Final Code\Growth_JY_Final/SCC_mat_Cumu_worst_GrowthNoAmb_1e9.mat')
worst_guess = worst_['v0']
check = loadmat(r'C:\Users\jiamingwang\Dropbox\share with John\Final Code/Growth_JY_Final/HJB_Growth_Neutral')

v0_guess = check['out_comp']
q_guess = check['q']
e_guess = check['e_hat']

In [21]:
smart_guess = {}
# prelin_results = {}
key = 'GrowthNeutral'
smart_guess = {}
smart_guess['v0'] = v0_guess
smart_guess['q'] = q_guess
smart_guess['e'] = e_guess
smart_guess['base'] = base_guess
smart_guess['worst'] = worst_guess
with open('./data/{}.pickle'.format(key + 'guess'), "wb") as file_:
    pickle.dump(smart_guess, file_, -1)
# smart_guess['WeightedNeutral'] = {}
# smart_guess['WeightedNeutral']['v0'] = v0_guess
# smart_guess['WeightedNeutral']['q'] = q_guess
# smart_guess['WeightedNeutral']['e'] = e_guess
# smart_guess['WeightedNeutral']['base'] = base_guess
# smart_guess['WeightedNeutral']['worst'] = worst_guess

In [ ]:
smart_guess.keys()

In [ ]:
with open('./data/smartguesses.pickle', "wb") as file_:
    pickle.dump(smart_guess, file_, -1)
    
# m1 = pickle.load(open('./data/comppref.pickle', "rb", -1))

In [ ]:
pickle.load(open('./data/smartguesses.pickle', "rb", -1))['WeightedAverse'].keys()

In [ ]:
def densityPlot(key = 'Weighted'):
    years = [50, 75, 100]

    titles = ["Year {}".format(year) for year in years]

    fig = make_subplots(1, len(years), print_grid = False, subplot_titles = titles)

    dom =beta_f_space
    inds = ((dom>=0) & (dom<=5e-3))

    for i, year in enumerate(years):
        # data = loadmat("{}/50-50 weight/Dist_{}yr.mat".format(quad_rule, year))
        data = Dists
        if key == 'Weighted': 
            if i == 0:
                fig.add_scatter(x = dom[inds] * 1000, y = data['Original'][inds], row = 1, col = i + 1,
                    name = 'Original Distribution', line = dict(color = '#1f77b4', width = 3), showlegend = True, legendgroup = 'Original Distribution')
                fig.add_scatter(x = dom[inds] * 1000, y = data['Nordhaus_year' + str(year)][inds], row = 1, col = i + 1,
                    name = 'Low Damage Function', line = dict(color = 'red', dash='dashdot', width = 3), showlegend = True, legendgroup = 'Low Damage Function')
                fig.add_scatter(x = dom[inds] * 1000, y = data['Weitzman_year' + str(year)][inds], row = 1, col = i + 1,
                    name = 'High Damage Function', line = dict(color = 'green', dash='dash', width = 3), showlegend = True, legendgroup = 'High Damage Function')
            else:
                fig.add_scatter(x = dom[inds] * 1000, y = data['Original'][inds], row = 1, col = i + 1,
                    name = 'Original Distribution', line = dict(color = '#1f77b4', width = 3), showlegend = False, legendgroup = 'Original Distribution')
                fig.add_scatter(x = dom[inds] * 1000, y = data['Nordhaus_year' + str(year)][inds], row = 1, col = i + 1,
                    name = 'Low Damage Function', line = dict(color = 'red', dash='dashdot', width = 3), showlegend = False, legendgroup = 'Low Damage Function')
                fig.add_scatter(x = dom[inds] * 1000, y = data['Weitzman_year' + str(year)][inds], row = 1, col = i + 1,
                    name = 'High Damage Function', line = dict(color = 'green', dash='dash', width = 3), showlegend = False, legendgroup = 'High Damage Function')

        elif key == 'High':
            if i == 0:
                fig.add_scatter(x = dom[inds] * 1000, y = data['Original'][inds], row = 1, col = i + 1,
                    name = 'Original Distribution', line = dict(color = '#1f77b4', width = 3), showlegend = True, legendgroup = 'Original Distribution')
                fig.add_scatter(x = dom[inds] * 1000, y = data['Weitzman_year' + str(year)][inds], row = 1, col = i + 1,
                    name = 'High Damage Function', line = dict(color = 'green', dash='dash', width = 3), showlegend = True, legendgroup = 'High Damage Function')
            else:
                fig.add_scatter(x = dom[inds] * 1000, y = data['Original'][inds], row = 1, col = i + 1,
                    name = 'Original Distribution', line = dict(color = '#1f77b4', width = 3), showlegend = False, legendgroup = 'Original Distribution')
                fig.add_scatter(x = dom[inds] * 1000, y = data['Weitzman_year' + str(year)][inds], row = 1, col = i + 1,
                    name = 'High Damage Function', line = dict(color = 'green', dash='dash', width = 3), showlegend = False, legendgroup = 'High Damage Function')


        elif key == 'Low':
            if i == 0:
                fig.add_scatter(x = dom[inds] * 1000, y = data['Original'][inds], row = 1, col = i + 1,
                    name = 'Original Distribution', line = dict(color = '#1f77b4', width = 3), showlegend = True, legendgroup = 'Original Distribution')
                fig.add_scatter(x = dom[inds] * 1000, y = data['Nordhaus_year' + str(year)][inds], row = 1, col = i + 1,
                    name = 'Low Damage Function', line = dict(color = 'red', dash='dashdot', width = 3), showlegend = True, legendgroup = 'Low Damage Function')
            else:
                fig.add_scatter(x = dom[inds] * 1000, y = data['Original'][inds], row = 1, col = i + 1,
                    name = 'Original Distribution', line = dict(color = '#1f77b4', width = 3), showlegend = False, legendgroup = 'Original Distribution')
                fig.add_scatter(x = dom[inds] * 1000, y = data['Nordhaus_year' + str(year)][inds], row = 1, col = i + 1,
                    name = 'Low Damage Function', line = dict(color = 'red', dash='dashdot', width = 3), showlegend = False, legendgroup = 'Low Damage Function')

    fig['layout'].update(title = key + " Damage Specification", showlegend = True, titlefont = dict(size = 20), height = 400)

    for i in range(len(years)):

        fig['layout']['yaxis{}'.format(i+1)].update(showgrid = False)
        fig['layout']['xaxis{}'.format(i+1)].update(showgrid = False)

    fig['layout']['yaxis1'].update(title=go.layout.yaxis.Title(
                                    text="Probability Density", font=dict(size=16)))
    fig['layout']['xaxis2'].update(title=go.layout.xaxis.Title(
                                    text="Climate Sensitivity", font=dict(size=16)), showgrid = False)

    fig = go.FigureWidget(fig)
    iplot(fig)

In [ ]:
densityPlot(damageSpec)

In [ ]:
def SCCDecomposePlot(key = 'Weighted'):

    if key == 'Low':

        data = SCCs
        x1, y1, x2, y2, x3, y3 = 60, 195, 93, 330, 96, 100

    elif key == 'Weighted':

        data = SCCs
        x1, y1, x2, y2, x3, y3 = 60, 320, 80, 315, 90, 350

    elif key == 'High':

        data = SCCs
        x1, y1, x2, y2, x3, y3 = 60, 340, 93, 495, 96, 430


    total_SCC = np.array(data['SCC'])
    external_SCC = np.array(data['SCC2'])
    uncertainty_SCC = np.array(data['SCC3'])
    private_SCC = np.array(data['SCC1'])
    x = np.linspace(0,100,400)

    total = go.Scatter(x = x, y = total_SCC,
                   name = 'Total', line = dict(color = '#1f77b4', dash = 'solid', width = 3),\
                       showlegend = False)
    external = go.Scatter(x = x, y = external_SCC,
                   name = 'Ambiguity', line = dict(color = 'red', dash = 'dot', width = 3),\
                          showlegend = False)
    uncertainty = go.Scatter(x = x, y = uncertainty_SCC,
                   name = 'No Ambiguity', line = dict(color = 'green', dash = 'dashdot', width = 3),\
                             showlegend = False)
    private = go.Scatter(x = x, y = private_SCC,
                   name = 'Private', line = dict(color = 'black', width = 3),\
                         showlegend = False)

    annotations=[dict(x=x1, y=y1, text="Total", textangle=0, ax=-100,
                ay=-75, font=dict(color="black", size=12), arrowcolor="black",
                arrowsize=3, arrowwidth=1, arrowhead=1),

                dict(x=x2, y=y2, text="Ambiguity", textangle=0, ax=-100,
                ay=0, font=dict(color="black", size=12), arrowcolor="black",
                arrowsize=3, arrowwidth=1, arrowhead=1),

                dict(x=x3, y=y3, text="No Ambiguity", textangle=0, ax=-80,
                ay=80, font=dict(color="black", size=12), arrowcolor="black",
                arrowsize=3, arrowwidth=1, arrowhead=1)]

    layout = dict(title = 'Social Cost of Carbon, {} Damage Specification'.format(key),
                  titlefont = dict(size = 20),
                  xaxis = go.layout.XAxis(title=go.layout.xaxis.Title(
                                    text='Years', font=dict(size=16)),
                                         tickfont=dict(size=12), showgrid = False),
                  yaxis = go.layout.YAxis(title=go.layout.yaxis.Title(
                                    text='Dollars per Ton of Carbon', font=dict(size=16)),
                                         tickfont=dict(size=12), showgrid = False), 
                  annotations=annotations
                  )

    fig = dict(data = [total, external, uncertainty], layout = layout)
    iplot(fig)
    

In [ ]:
SCCDecomposePlot(damageSpec)

In [ ]:
def emissionPlot(damageSpec, ξ):

    colors = {'High': 'red', 'Low': 'green', 'Weighted': '#1f77b4'}
    lines = {'Averse': 'solid', "Neutral": 'dashdot'}

    # damageSpecs = ['High', 'Low', 'Weighted']
    # aversionSpecs = ['Averse', 'Neutral']
    # colors = ['green', '#1f77b4', 'red']
    # lines = ['solid', 'dashdot'] 

    x = np.linspace(0, 100, 400)
    data = []

    data.append(go.Scatter(x = x, y = e_hists[:,0], name = damageSpec +  ' Damage w/ ξ= {}'.format(ξ),
        line = dict(width = 2, dash = 'solid', color = colors[damageSpec]), showlegend = True))

    layout = dict(title = 'Emissions Plot with {} Damage Setting, ξ = {}'.format(damageSpec, ξ),
      titlefont = dict(size = 20),
      xaxis = go.layout.XAxis(title=go.layout.xaxis.Title(
                        text='Years', font=dict(size=16)),
                             tickfont=dict(size=12), showgrid = False, showline = True),
      yaxis = go.layout.YAxis(title=go.layout.yaxis.Title(
                        text='Gigatons of Carbon', font=dict(size=16)),
                             tickfont=dict(size=12), showgrid = False),
      legend = dict(orientation = 'h', y = 1.15)
      )

    fig = dict(data = data, layout = layout)
    iplot(fig)

In [ ]:
emissionPlot(damageSpec, ξp)